# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 14 2023 11:59AM,259271,19,OPUS-105692,Opus Life Sciences,Released
1,Mar 14 2023 11:01AM,259267,10,9265147,MedStone Pharma LLC,Released
2,Mar 14 2023 11:01AM,259267,10,9265148,MedStone Pharma LLC,Executing
3,Mar 14 2023 11:01AM,259267,10,9265149,MedStone Pharma LLC,Released
4,Mar 14 2023 11:01AM,259267,10,9265151,MedStone Pharma LLC,Released
5,Mar 14 2023 11:01AM,259267,10,9265152,MedStone Pharma LLC,Released
6,Mar 14 2023 11:01AM,259267,10,9265156,MedStone Pharma LLC,Released
7,Mar 14 2023 11:01AM,259267,10,9265193,MedStone Pharma LLC,Released
8,Mar 14 2023 11:01AM,259267,10,9265187,MedStone Pharma LLC,Released
9,Mar 14 2023 11:01AM,259267,10,9265188,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,259265,Executing,2
17,259265,Released,2
18,259267,Executing,10
19,259267,Released,21
20,259271,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259260,NaN,6.0,11.0
259263,NaN,NaN,1.0
259265,NaN,2.0,2.0
259267,NaN,10.0,21.0
259271,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259195,0.0,45.0,19.0
259215,0.0,0.0,10.0
259221,11.0,11.0,0.0
259225,0.0,0.0,1.0
259243,15.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259195,0,45,19
259215,0,0,10
259221,11,11,0
259225,0,0,1
259243,15,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259195,0,45,19
1,259215,0,0,10
2,259221,11,11,0
3,259225,0,0,1
4,259243,15,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259195,,45,19
1,259215,,,10
2,259221,11,11,
3,259225,,,1
4,259243,15,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 14 2023 11:59AM,259271,19,Opus Life Sciences
1,Mar 14 2023 11:01AM,259267,10,MedStone Pharma LLC
32,Mar 14 2023 10:57AM,259265,10,Emerginnova
36,Mar 14 2023 10:56AM,259263,10,"Nextsource Biotechnology, LLC"
37,Mar 14 2023 10:39AM,259259,19,"AdvaGen Pharma, Ltd"
54,Mar 14 2023 10:32AM,259260,10,Eywa Pharma Inc.
71,Mar 14 2023 10:19AM,259258,19,"NAPP Technologies, LLC"
72,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc."
82,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc."
146,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 14 2023 11:59AM,259271,19,Opus Life Sciences,,,1
1,Mar 14 2023 11:01AM,259267,10,MedStone Pharma LLC,,10,21
2,Mar 14 2023 10:57AM,259265,10,Emerginnova,,2,2
3,Mar 14 2023 10:56AM,259263,10,"Nextsource Biotechnology, LLC",,,1
4,Mar 14 2023 10:39AM,259259,19,"AdvaGen Pharma, Ltd",,3,14
5,Mar 14 2023 10:32AM,259260,10,Eywa Pharma Inc.,,6,11
6,Mar 14 2023 10:19AM,259258,19,"NAPP Technologies, LLC",,,1
7,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",,,10
8,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",,45,19
9,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC",,1,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 11:59AM,259271,19,Opus Life Sciences,1,,
1,Mar 14 2023 11:01AM,259267,10,MedStone Pharma LLC,21,10,
2,Mar 14 2023 10:57AM,259265,10,Emerginnova,2,2,
3,Mar 14 2023 10:56AM,259263,10,"Nextsource Biotechnology, LLC",1,,
4,Mar 14 2023 10:39AM,259259,19,"AdvaGen Pharma, Ltd",14,3,
5,Mar 14 2023 10:32AM,259260,10,Eywa Pharma Inc.,11,6,
6,Mar 14 2023 10:19AM,259258,19,"NAPP Technologies, LLC",1,,
7,Mar 14 2023 9:48AM,259215,15,"Alliance Pharma, Inc.",10,,
8,Mar 14 2023 9:46AM,259195,15,"Alliance Pharma, Inc.",19,45,
9,Mar 14 2023 9:07AM,259245,10,"Digital Brands, LLC",9,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 11:59AM,259271,19,Opus Life Sciences,1,,
1,Mar 14 2023 11:01AM,259267,10,MedStone Pharma LLC,21,10,
2,Mar 14 2023 10:57AM,259265,10,Emerginnova,2,2,
3,Mar 14 2023 10:56AM,259263,10,"Nextsource Biotechnology, LLC",1,,
4,Mar 14 2023 10:39AM,259259,19,"AdvaGen Pharma, Ltd",14,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 11:59AM,259271,19,Opus Life Sciences,1.0,NaN,NaN
1,Mar 14 2023 11:01AM,259267,10,MedStone Pharma LLC,21.0,10.0,NaN
2,Mar 14 2023 10:57AM,259265,10,Emerginnova,2.0,2.0,NaN
3,Mar 14 2023 10:56AM,259263,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Mar 14 2023 10:39AM,259259,19,"AdvaGen Pharma, Ltd",14.0,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 11:59AM,259271,19,Opus Life Sciences,1.0,0.0,0.0
1,Mar 14 2023 11:01AM,259267,10,MedStone Pharma LLC,21.0,10.0,0.0
2,Mar 14 2023 10:57AM,259265,10,Emerginnova,2.0,2.0,0.0
3,Mar 14 2023 10:56AM,259263,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Mar 14 2023 10:39AM,259259,19,"AdvaGen Pharma, Ltd",14.0,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1555543,44.0,20.0,15.0
15,518410,29.0,45.0,0.0
19,1037009,16.0,14.0,11.0
20,259225,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1555543,44.0,20.0,15.0
1,15,518410,29.0,45.0,0.0
2,19,1037009,16.0,14.0,11.0
3,20,259225,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,44.0,20.0,15.0
1,15,29.0,45.0,0.0
2,19,16.0,14.0,11.0
3,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,15,Released,29.0
2,19,Released,16.0
3,20,Released,1.0
4,10,Executing,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,15.0,0.0,11.0,0.0
Executing,20.0,45.0,14.0,0.0
Released,44.0,29.0,16.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,15.0,0.0,11.0,0.0
1,Executing,20.0,45.0,14.0,0.0
2,Released,44.0,29.0,16.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,15.0,0.0,11.0,0.0
1,Executing,20.0,45.0,14.0,0.0
2,Released,44.0,29.0,16.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()